In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model, metrics, cross_validation

df = pd.read_csv("data/train.csv")

    
X_matrix = []
y_vector = []

for row in df.values:
    X_matrix.append(row[1:785])
    y_vector.append(row[:1])
   
X_matrix = np.array(X_matrix)
y = np.array(y_vector)

X = np.zeros((len(X_matrix), len(X_matrix[0])))

#means = np.mean(X_matrix, axis=1)
#stds = np.std(X_matrix, axis=1)
#mean = np.mean(means, axis=0)
#std = np.mean(stds, axis=0)
#print mean, std    


/home/stepan/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
minimums = list()
maximums = list()

for i in np.arange(0, len(X_matrix)):
    minimums.append(min(X_matrix[i]))
    maximums.append(max(X_matrix[i]))
    
maxX = max(maximums)
minX = min(minimums)

In [3]:
for i in np.arange(0, len(X)):
 for j in np.arange(0, len(X[i])):
    X[i][j] = float(X_matrix[i][j] - minX) / (maxX - minX)

column = np.ones((1, len(X)))
column = np.reshape(column, (len(X), 1))
X = np.hstack((column, X))    

print X[0]

[ 1.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.

In [7]:
train_data, test_data, train_labels, test_labels = cross_validation.train_test_split(X, y, test_size=0.3)
#разбили наши данные на обучающие и тестовые (7 к 3)

In [8]:
print len(X)

def getPredict(probabilities):
    length = len(probabilities[0])
    new_probs = list()
    
    for j in np.arange(0, length):
        max_proba = 0
        label = 0
        proba = list()
        for i in range(0, 10):
            proba.append(probabilities[i][j][1])
        new_probs.append(proba)
    
    result_predict = list() #результат работы 10 регрессоров (предсказание)
    for j in np.arange(0, length):
        max_proba = 0
        label = 0
        for i in range(0, 10):
            if new_probs[j][i] >= max_proba:
                max_proba = new_probs[j][i]
                label = i
        result_predict.append(label)
    return result_predict

42000


In [41]:
#учим 10 логистических регрессоров выявлять цифры от 0 до 9

#загрузим тестовые данные 
#test_data_all = pd.read_csv("trainingandtestdata/test.csv")
log_regressors = list()#список регрессоров

def getProbabilitiesWithTrain( log_regressors, train_data, test_data, train_labels, test_labels):
    probs = list() #список списков вероятностей для каждого регрессора
    accuracy = list() #список результатов точности для каждого регрессор
    for j in range(0, 10):
        train_labels_j = np.zeros((len(train_labels), 1))
        test_labels_j = np.zeros((len(test_labels), 1))
        
        for i in np.arange(0, len(train_labels)):
            if train_labels[i][0] == j:
                train_labels_j[i] = [1]
            else:
                train_labels_j[i] = [0]
        
        for i in np.arange(0, len(test_labels)):
            if test_labels[i][0] == j:
                test_labels_j[i] = [1]
            else:
                test_labels_j[i] = [0]
                
        log_regressor = linear_model.LogisticRegression(random_state=1, penalty='l1', max_iter=10000)
        log_regressor.fit(train_data, train_labels_j)
        predict = log_regressor.predict(test_data)
        predict_proba = log_regressor.predict_proba(test_data)
        probs.append(predict_proba)
        log_regressors.append(log_regressor)
        accuracy.append(metrics.accuracy_score(test_labels_j, predict))
    return probs


def getProbabilities( log_regressors, test_data):
    probs = list() #список списков вероятностей для каждого регрессора
    accuracy = list() #список результатов точности для каждого регрессор
    for j in range(0, 10):
        predict = log_regressors[j].predict(test_data)
        predict_proba = log_regressors[j].predict_proba(test_data)
        probs.append(predict_proba)
    return probs
#print accuracy
#rint probs[:2]

In [42]:
#получили список вероятностей для каждого регрессора - probs. посмотрим на него - будем принимать решение,
#исходя из максимальной вероятности на каждом объекте
probs = getProbabilitiesWithTrain(log_regressors, train_data, test_data, train_labels, test_labels)
print probs[:3]

accuracy = metrics.accuracy_score(test_labels, getPredict(probs)) #
print accuracy
#print new_probs[:5]
#print result_predict[:10]
#print test_labels[:10]

#print accuracy


#видим, что результат довольно неплохой: ошибка около 9%


[array([[  9.99927699e-01,   7.23012512e-05],
       [  9.99998967e-01,   1.03341619e-06],
       [  9.99459134e-01,   5.40866469e-04],
       ..., 
       [  9.99999980e-01,   1.99192318e-08],
       [  9.99999492e-01,   5.08307318e-07],
       [  9.99999173e-01,   8.26723734e-07]]), array([[  9.99998780e-01,   1.22015249e-06],
       [  9.81367165e-02,   9.01863283e-01],
       [  9.99998626e-01,   1.37397113e-06],
       ..., 
       [  9.99999864e-01,   1.35900621e-07],
       [  2.51452489e-01,   7.48547511e-01],
       [  9.99999772e-01,   2.28391200e-07]]), array([[  9.99940009e-01,   5.99909344e-05],
       [  9.98406846e-01,   1.59315441e-03],
       [  9.99995770e-01,   4.23044446e-06],
       ..., 
       [  9.99889680e-01,   1.10319815e-04],
       [  8.24746354e-01,   1.75253646e-01],
       [  9.98341377e-01,   1.65862301e-03]])]
0.911984126984


In [43]:
test_data_all = pd.read_csv("data/test.csv") #загрузим тестовую выборку для отправки результата первой модели

In [44]:
X_matrix = []

for row in test_data_all.values:
    X_matrix.append(row[0:784])
   
X_matrix = np.array(X_matrix)

X = np.zeros((len(X_matrix), len(X_matrix[0])))


In [45]:
minimums = list()
maximums = list()

for i in np.arange(0, len(X_matrix)):
    minimums.append(min(X_matrix[i]))
    maximums.append(max(X_matrix[i]))
    
maxX = max(maximums)
minX = min(minimums)

for i in np.arange(0, len(X)):
 for j in np.arange(0, len(X[i])):
    X[i][j] = float(X_matrix[i][j] - minX) / (maxX - minX)

column = np.ones((1, len(X)))
column = np.reshape(column, (len(X), 1))
X = np.hstack((column, X))    

print X[0]

[ 1.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.

In [46]:
predict_all = getPredict(getProbabilities(log_regressors, X))

In [52]:
print predict_all[:200]

f = open('competition.csv','w')
f.write('ImageId,Label\n')

for i, label in enumerate(predict_all):
    s = str(i+1)+ ',' + str(label) + '\n'
    f.write(s)
f.close()
    
#получился очень даже неплохой результат - ошибка меньше 9%, но стоит подумать, как можно повысить качество
#

[2, 0, 9, 7, 3, 7, 0, 3, 0, 3, 5, 7, 4, 0, 4, 3, 3, 1, 9, 0, 9, 1, 1, 5, 7, 4, 2, 7, 7, 7, 7, 5, 4, 2, 6, 2, 9, 5, 1, 6, 7, 7, 4, 9, 8, 7, 8, 8, 6, 7, 6, 8, 8, 3, 8, 2, 1, 2, 2, 0, 4, 1, 7, 0, 0, 0, 1, 9, 0, 1, 6, 5, 8, 8, 2, 8, 3, 9, 2, 3, 5, 9, 1, 0, 9, 2, 4, 3, 6, 7, 2, 0, 6, 6, 8, 4, 3, 9, 7, 7, 0, 3, 2, 0, 7, 3, 0, 5, 0, 3, 0, 0, 4, 7, 1, 7, 1, 1, 5, 3, 3, 7, 2, 8, 6, 3, 8, 7, 8, 4, 3, 5, 6, 0, 0, 0, 3, 1, 5, 6, 4, 3, 4, 5, 5, 8, 7, 7, 2, 8, 4, 3, 5, 6, 5, 3, 7, 3, 7, 8, 3, 0, 4, 5, 1, 8, 7, 6, 4, 0, 2, 7, 8, 6, 1, 3, 7, 4, 1, 2, 4, 3, 8, 2, 4, 9, 2, 1, 6, 0, 6, 1, 4, 9, 6, 0, 9, 7, 6, 9]


In [14]:
print predict[:10]
accuracy = metrics.accuracy_score(test_labels, predict)
print accuracy

[6 7 5 7 6 8 5 5 4 4]
0.912619047619


In [15]:
predict_all = log_r.predict(X)
predict_all_proba = log_r.predict_proba(X)

print predict_all[:10]
print predict_all_proba[:10]

[1 0 1 4 0 0 7 3 5 3]
[[  3.80373460e-07   9.45358281e-01   8.65377777e-03   1.37596198e-03
    2.93084016e-03   2.45688934e-03   3.30132894e-04   2.68637173e-04
    3.85964438e-02   2.86556500e-05]
 [  9.87340208e-01   6.04800066e-15   6.34478981e-04   1.74811884e-08
    9.72629141e-11   2.65113515e-07   7.48328845e-04   3.27088932e-08
    1.12761698e-02   4.99351904e-07]
 [  3.94467101e-09   9.15758234e-01   1.52396578e-02   3.99440153e-02
    5.04953050e-04   6.60802103e-04   2.78421330e-03   1.07647217e-02
    2.26020958e-04   1.41173776e-02]
 [  1.29689317e-02   4.89638669e-08   2.29723402e-01   7.89648457e-04
    5.59239292e-01   1.09251616e-01   5.66590079e-02   2.77039493e-03
    1.05389429e-03   2.75437633e-02]
 [  9.86216453e-01   5.60335369e-18   3.71072719e-03   2.88350767e-09
    4.67223921e-14   2.10804178e-08   6.21280782e-07   1.93422764e-15
    1.00721742e-02   3.79689477e-13]
 [  9.33518286e-01   8.67814589e-12   4.85253922e-03   2.20604318e-04
    3.34594216e-05   5.